In [1]:
print("ok")

ok


In [2]:
%pwd

'/Users/palomavaldes/Desktop/VitaCare/VitaCare/research'

In [3]:
import os
os.chdir('../')

In [4]:
%pwd

'/Users/palomavaldes/Desktop/VitaCare/VitaCare'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# extract the data from the pdf file in the folder 'data'
def loader_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [7]:
extracted_data = loader_pdf_file(data = 'data/')

Ignoring wrong pointing object 143 0 (offset 0)
Ignoring wrong pointing object 194 0 (offset 0)
Ignoring wrong pointing object 6539 0 (offset 0)
Ignoring wrong pointing object 8120 0 (offset 0)
Ignoring wrong pointing object 8128 0 (offset 0)
Ignoring wrong pointing object 9479 0 (offset 0)
Ignoring wrong pointing object 9557 0 (offset 0)
Ignoring wrong pointing object 9565 0 (offset 0)
Ignoring wrong pointing object 22389 0 (offset 0)
Ignoring wrong pointing object 22939 0 (offset 0)
Ignoring wrong pointing object 23291 0 (offset 0)
Ignoring wrong pointing object 27119 0 (offset 0)
Ignoring wrong pointing object 28583 0 (offset 0)
Ignoring wrong pointing object 34442 0 (offset 0)
Ignoring wrong pointing object 34567 0 (offset 0)
Ignoring wrong pointing object 35529 0 (offset 0)
Ignoring wrong pointing object 35832 0 (offset 0)
Ignoring wrong pointing object 37033 0 (offset 0)
Ignoring wrong pointing object 37210 0 (offset 0)
Ignoring wrong pointing object 40370 0 (offset 0)
Ignoring w

In [8]:
# extracted_data

In [9]:
# split the data (the pdf file) into text chuncks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("Length of text chunks:", len(text_chunks))

Length of text chunks: 40324


In [11]:
# text_chunks

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
# download  the embeddings from Hugging Face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [14]:
pip install sentence-transformers==2.2.2 huggingface_hub==0.15.1

  Using cached huggingface_hub-0.15.1-py3-none-any.whl.metadata (8.0 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-macosx_10_11_x86_64.whl.metadata (6.7 kB)
Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
Using cached tokenizers-0.13.3-cp310-cp310-macosx_10_11_x86_64.whl (4.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: huggingface_hub━━━━━━━━━ 0/2 [tokenizers]
    Found existing installation: huggingface-hub 0.33.2/2 [tokenizers]
    Uninstalling huggingface-hub-0.33.2:━━━━ 0/2 [tokenizers]
      Successfully uninstalled huggingface-hub-0.33.2m0/2 [tokenizers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [huggingface_hub] [huggingface_hub]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
co

In [15]:
pip install "numpy<2"

Note: you may need to restart the kernel to use updated packages.


In [16]:
embeddings = download_hugging_face_embeddings()

/var/folders/34/0t4x99w11mn4pn5llpjnbprr0000gn/T/ipykernel_75094/3816888157.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
/opt/anaconda3/envs/llmpp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/llmpp/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils.

In [17]:
query_result = embeddings.embed_query("hello world")
print("length:", len(query_result))

length: 384


In [18]:
# query_result

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
import os
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
CO_API_KEY = os.environ.get('CO_API_KEY')

In [21]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "vitacare"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )


In [31]:
import os 
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["CO_API_KEY"] = CO_API_KEY

In [23]:
# Embed each chunk and upsert the embeddings into Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [24]:
# load existing index
from langchain_pinecone import PineconeVectorStore

#embed each chunk and upsert the embeddings into Pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='1f8b815a-25ee-4e79-abc9-27d528b4160e', metadata={'author': 'Paloma Valdes', 'creationdate': "D:20250708022456Z00'00'", 'creator': 'Safari', 'moddate': "D:20250708022456Z00'00'", 'page': 55.0, 'page_label': '56', 'producer': 'macOS Version 14.6.1 (Build 23G93) Quartz PDFContext', 'source': 'data/medical_book.pdf', 'title': 'https:/staibabussalamsula.ac.id/wp-content/uploads/2024/06/The-Gale-Encyclopedia-of-Medicine-3rd-Edition-staibabussalamsula.ac_.id_', 'total_pages': 4505.0}, page_content='Isotretinoin—Ad r u gt h a td e c r e a s e ss e b u mp r o d u c -\ntion and dries up acne pimples.\nSebaceous follicles—As t r u c t u r ef o u n dw i t h i nt h e\nskin that houses the oil-producing glands and hair\nfollicles, where pimples form.\nSebum— An oily skin moisturizer produced by\nsebaceous glands.\nTretinoin—Ad r u gt h a tw o r k sb yi n c r e a s i n gt h e\nturnover (death and replacement) of skin cells.\nAcne vulgaris affecting a woman’s face. Acne is the general'),

In [32]:
from langchain_cohere import ChatCohere
llm = ChatCohere(
    model="command-r-plus",  # Their best model
    temperature=0.3,
    max_tokens=500
)

In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [36]:
response = rag_chain.invoke({"input": "what is gigantism?"})
print(response["answer"])

Gigantism is a condition caused by excessive growth, often due to an abnormal release of chemicals from the pituitary gland. It results in accelerated growth, with patients exhibiting enlarged hands, feet, ears, and noses, as well as a protruding jaw and long face. It is distinct from acromegaly, as gigantism occurs during life, not just in the hands and feet.
